# Prepare Environment (default operations)

In [ ]:
panikos = True

In [1]:
import os, tensorflow as tf, json, matplotlib.pyplot as plt

if panikos:
    from data_loader.hgf_export_Pan import HGFresource
else:
    from data_loader.hgf_export import HGFresource

from dotenv import load_dotenv

In [2]:
load_dotenv()
HGF_TOKEN = os.environ['HUGGINGFACE_TOKEN']
HGF_DATA_REPO = os.environ['HUGGINGFACE_DATASET_V2_REPO']
HGF_BASELINE_MODEL_REPO = os.environ['HUGGINGFACE_BASELINE_CNN_REPO']
HGF_BASELINE_MODEL_CONFIG = os.environ['HUGGINGFACE_BASELINE_CNN_CONFIG_FILE']
HGF_BASELINE_MODEL_WEIGHTS = os.environ['HUGGINGFACE_BASELINE_CNN_WEIGHTS_FILE']

In [3]:
hgf = HGFresource(token=HGF_TOKEN)

In [4]:
# load config of baseline model which we need to optimize
model_config_file = hgf.load_file(repo=HGF_BASELINE_MODEL_REPO, filename=HGF_BASELINE_MODEL_CONFIG)
with open(model_config_file, 'r') as f:
    model_config = f.read()
model_config = json.loads(model_config)
# extract number of classes from the last layer
N_CLASSES = model_config['layers'][-1]['config']['units']

# # alternatively, you can load the full baseline model with pre-defined weights
# # you may want to do this for the sake of performance comparison
# # but note that you will occupy additional RAM space (without a good reason probably)
baseline_model = hgf.load_model(
    repo=HGF_BASELINE_MODEL_REPO,
    filename={
        'model_weights': HGF_BASELINE_MODEL_WEIGHTS,
        'model_config': HGF_BASELINE_MODEL_CONFIG
    }
)
# # extract number of classes from the last layer
# N_CLASSES = baseline_model.layers[-1].get_config()['units']

In [5]:
train_data, test_data = hgf.load_data_tfds(repo=HGF_DATA_REPO, n_classes=N_CLASSES, batch_size=32)

Map:   0%|          | 0/13663 [00:00<?, ? examples/s]

In [9]:
# learning rate may require fine-tuning but use the optimizer specified below
# OPTIMIZER = tf.keras.optimizers.legacy.Adam(learning_rate=0.001) # use this if you have M1/M2 Mac, otherwise use the next line
# OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)
LOSS = 'categorical_crossentropy'

if panikos:
    METRICS = [tf.keras.metrics.CategoricalAccuracy()]
else:
    METRICS = [tf.keras.metrics.F1Score('weighted')]
    
EPOCHS = 20
INPUT_SHAPE = (256, 219, 3)

In [10]:
# initialize the baseline model architecture
model = tf.keras.Sequential.from_config(model_config)
# view the model architecture
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 217, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 127, 108, 16)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 106, 32)      4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 62, 53, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 60, 51, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 30, 25, 64)      

In [15]:
model.compile(loss=LOSS,metrics=METRICS)

In [11]:
def history_plot(history):
    train_f1_scores = history.history['f1_score']
    val_f1_scores = history.history['val_f1_score']

    epochs = range(1, len(train_f1_scores) + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_f1_scores, 'bo-', label='Training F1 Score')
    plt.plot(epochs, val_f1_scores, 'ro-', label='Validation F1 Score')
    plt.title('Training and Validation F1 Score')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    plt.show()

# Your Code Starts Here

In [ ]:
# some fancy stuff

In [16]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=EPOCHS
)

Epoch 1/20
1281/1281 [==============================] - 189s 115ms/step - loss: 1.6341 - categorical_accuracy: 0.6310 - val_loss: 0.9419 - val_categorical_accuracy: 0.7606
Epoch 2/20
1281/1281 [==============================] - 135s 105ms/step - loss: 0.6529 - categorical_accuracy: 0.8258 - val_loss: 0.6182 - val_categorical_accuracy: 0.8363
Epoch 3/20
1281/1281 [==============================] - 133s 104ms/step - loss: 0.4607 - categorical_accuracy: 0.8716 - val_loss: 0.5385 - val_categorical_accuracy: 0.8574
Epoch 4/20
1281/1281 [==============================] - 166s 129ms/step - loss: 0.3721 - categorical_accuracy: 0.8910 - val_loss: 0.5276 - val_categorical_accuracy: 0.8534
Epoch 5/20
1281/1281 [==============================] - 150s 117ms/step - loss: 0.3212 - categorical_accuracy: 0.9033 - val_loss: 0.5081 - val_categorical_accuracy: 0.8653
Epoch 6/20
1281/1281 [==============================] - 140s 109ms/step - loss: 0.2866 - categorical_accuracy: 0.9132 - val_loss: 0.5094 - v

# Commit Your Results to Hugging Face Once Done

In [ ]:
# NOTE: when naming your weights file
# indicate the type of model
# e.g.:
## 'tuned_optimizer_model_weights.h5'
## 'data_augmentation_model_weights.h5'
WEIGHTS_TO_COMMIT = '<your_name>_model_weights.h5'
# save model weights
model.save_weights(WEIGHTS_TO_COMMIT)

In [ ]:
PERFORMANCE_TO_COMMIT = '<your_name_same_as_for_weights>_model_performance.json'


if panikos:
       performance_dict = {
        'train': {
            'weighted_f1': history.history['categorical_accuracy'][-1]
        },
        'test': {
            'weighted_f1': history.history['val_categorical_accuracy'][-1]
            }
    }
else:
    performance_dict = {
        'train': {
            'weighted_f1': history.history['f1_score'][-1]
        },
        'test': {
            'weighted_f1': history.history['val_f1_score'][-1]
            }
    }
    


performance_dict = json.dumps(performance_dict)
with open(PERFORMANCE_TO_COMMIT, 'w') as f:
    f.write(performance_dict)

In [ ]:
hgf.commit_to_hub(
    repo=HGF_BASELINE_MODEL_REPO,
    path_on_local=[WEIGHTS_TO_COMMIT, PERFORMANCE_TO_COMMIT],
    path_in_repo=[WEIGHTS_TO_COMMIT, PERFORMANCE_TO_COMMIT],
    # do not forget to change your commit message
    # to add even more clarity
    commit_message='your commit message'
)

In [ ]:
# remove files from local = clean up
for path in [WEIGHTS_TO_COMMIT, PERFORMANCE_TO_COMMIT]:
    os.remove(path)